In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv', encoding='latin_1')
test  = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv', encoding='latin_1')
sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv', encoding='latin_1')

In [ ]:
import matplotlib
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder as LE


pd.set_option('display.max_columns', None)
sns.set_style("darkgrid")
print(f"Numpy version : Numpy {np.__version__}")
print(f"Pandas version : Pandas {pd.__version__}")
print(f"Matplotlib version : Matplotlib {matplotlib.__version__}")
print(f"Seaborn version : Seaborn {sns.__version__}")
print(f"SkLearn version : SkLearn {sklearn.__version__}")

In [ ]:
train

In [ ]:
train.describe()

## EDA

In [ ]:
def detailed(data):
    ddata = pd.DataFrame(index=['total', '% missing' , 'data Type', 'range'])
    
    for cols in data.columns:
        size = data[cols].size
        pnul=round(data[cols].isna().sum()/size,2)
        nul_val = f"{pnul*100}% - {data[cols].isna().sum()}/{size}"
        dtype = 'categorical' if data[cols].dtype == object else 'Numerical' 
        rng = f"{len(data[cols].unique())} labels"\
        if dtype == 'categorical' else\
        f"{round(data[cols].min(),2)}-{round(data[cols].max(),2)}"
        ddata[cols] = [size,nul_val,dtype,rng]
    return ddata

In [ ]:
detailed(train)

In [ ]:
detailed(test)

In [ ]:
train.drop(['id'],axis=1, inplace= True)
# test.drop(['id'], axis=1, inplace=True)

In [ ]:
train

In [ ]:
plt.figure(figsize=[20,10])
train['target'].plot.hist()

In [ ]:
plt.figure(figsize=[10,20])
train['target'].plot.box()

In [ ]:
# train.index

In [ ]:
train['target'].describe()

In [ ]:
def outlier_idx(data):
    
    Q1, Q3 = np.quantile(data,[0.25, 0.75])
    
    IQR = Q3-Q1
    
    UB = Q3 + (IQR * 1.5)
    LB = Q1 - (IQR * 1.5)
    
    return np.where((data>UB)|(data<LB))

In [ ]:
idx=outlier_idx(train['target'])

In [ ]:
train.drop(idx[0], inplace=True)

In [ ]:
train

## PRE-PROCESSING  

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def preprocess(d):
    data = d.copy()
    
    categories = ['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9']
    
    for col in categories:
        le = LE()
        data[col] = le.fit_transform(data[col])
#         test[col] = le.fit_transform(test[col])
    scaler = MinMaxScaler()
    if 'target' == data.columns[-1]:
        X = data[data.columns[:-1]]
    else:
        X = data
    X=scaler.fit_transform(X)
    return X

In [ ]:
X = preprocess(train)
y = train['target']

### Scaling 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2, shuffle=True, random_state=111 )

## BASIC MODELLING 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(x_train, y_train)

In [ ]:
y_preds = lr.predict(x_test)

In [ ]:
mean_absolute_error(y_test, y_preds)

In [ ]:
# preprocess(test[testt.col])

In [ ]:
target_X = preprocess(test[test.columns[1:]])

In [ ]:
sub['target']=lr.predict(target_X)

In [ ]:
sub.to_csv('submission.csv', index=False)